## Vectroize Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import numpy as np
import tensorflow_hub as hub

In [ ]:
data_dir = "/content/drive/MyDrive/sjsu/255 project/dataset/train/"
train_csv_df = pd.read_csv("/content/drive/MyDrive/sjsu/255 project/dataset/train.csv")

In [ ]:
train_csv_df['img_path'] = train_csv_df.apply(lambda row : data_dir + str(row['Id']) + '.jpg', axis=1)

In [ ]:
train_csv_df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,img_path
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,/content/drive/MyDrive/sjsu/255 project/datase...
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,/content/drive/MyDrive/sjsu/255 project/datase...
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,/content/drive/MyDrive/sjsu/255 project/datase...
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,/content/drive/MyDrive/sjsu/255 project/datase...
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,/content/drive/MyDrive/sjsu/255 project/datase...


In [ ]:
file_path = train_csv_df['img_path'].values
labels = train_csv_df['Pawpularity'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(file_path, labels, test_size=0.2, random_state=40)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=40)

In [ ]:
X_train.shape

(6739,)

In [ ]:
ds_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
ds_val = tf.data.Dataset.from_tensor_slices((X_val, y_val))
ds_test = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [ ]:
img_size = [380, 380]
maxDelta = 0.2
contrast = [0.2, 0.5]
batch_size = 256

# pipeline
# image decoder
def img_decoder(path, label):
  img_file = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img_file, channels=3)
  # resize images with pad to desired size
  img = tf.image.resize_with_pad(img, img_size[0], img_size[1])
  # image normalization
  img = tf.cast(img, tf.float32) / 255.0
  # image reshape
  img = tf.reshape(img, [*img_size, 3])
  
  return img, tf.cast(label, tf.float32) / 100.0

In [ ]:
ds_train = ds_train.map(img_decoder).batch(batch_size)
ds_val = ds_val.map(img_decoder).batch(batch_size)
ds_test = ds_test.map(img_decoder).batch(batch_size)

In [ ]:
contrast_factor = (0.2, 0.2)
ratation_factor = 0.2
# augmentation
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(ratation_factor),
  layers.RandomContrast(contrast_factor)
])

### Load Pre-trained EffNetB4 (with Base Model Frozen)


In [ ]:
IMAGE_HEIGHT, IMAGE_WIDTH = img_size[0], img_size[1]
efficientNetB4 = tf.keras.applications.EfficientNetB4(
    include_top=False, weights='imagenet',
    input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3), pooling='avg'
)


In [ ]:
efficientNetB4.summary()

Model: "efficientnetb4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 380, 380, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 380, 380, 3)  0           ['input_4[0][0]']                
                                                                                                  
 normalization_1 (Normalization  (None, 380, 380, 3)  7          ['rescaling_1[0][0]']            
 )                                                                                                
                                                                                     

In [ ]:
len(efficientNetB4.layers)

475

In [ ]:
# Unfreeze a little
layers_to_freeze = 460
for l in efficientNetB4.layers[:layers_to_freeze]:
  l.trainable = False

In [ ]:
model1 = tf.keras.Sequential([
  tf.keras.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH,3)),
  data_augmentation,
  efficientNetB4,
  layers.Dropout(0.3),
  tf.keras.layers.BatchNormalization(),
  layers.Dense(512, activation="elu"),
  layers.Dropout(0.2),
  tf.keras.layers.BatchNormalization(),
  layers.Dense(128, activation="elu"),
  layers.Dense(1)                     
])

In [ ]:
model1.load_weights("/content/drive/MyDrive/sjsu/255 project/code/zzj/EfficientNetB4/EfficientNetB4_improved/efficientNet_weights.hd5")

In [ ]:
def my_rmse(y_true, y_pred):
    #difference between true label and predicted label
    error = 100 * (y_true - y_pred)    
    #square of the error
    sqr_error = tf.square(error)
    #mean of the square of the error
    mean_sqr_error = tf.keras.backend.mean(sqr_error)
    #square root of the mean of the square of the error
    sqrt_mean_sqr_error = tf.sqrt(mean_sqr_error)
    #return the error
    return sqrt_mean_sqr_error


In [ ]:
decay_rate = 0.96
decay_steps = 1000
initial_learning_rate = 0.0001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True)

model1.compile(tf.keras.optimizers.Adam(learning_rate=lr_schedule),
            loss=my_rmse,
            metrics=[my_rmse])

fpath = "/content/drive/MyDrive/sjsu/255 project/code/zzj/EfficientNetB4_improved_fine_tune/efficientNet_fine_weights.hd5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
  filepath=fpath,
  save_weights_only=True,
  monitor='val_my_rmse',
  mode='min',
  save_best_only=True)

# early_stop = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss', patience=5, restore_best_weights=True)

# history1 = model1.fit(ds_train, epochs=500, validation_data=ds_val, callbacks=[model_checkpoint_callback, early_stop])

In [ ]:
model1.evaluate(ds_test)

8/8 [==============================] - 86s 8s/step - loss: 21.1573 - my_rmse: 21.1323


[21.15732192993164, 21.132261276245117]

In [ ]:
model1.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 380, 380, 3)       0         
                                                                 
 efficientnetb4 (Functional)  (None, 1792)             17673823  
                                                                 
 dropout_10 (Dropout)        (None, 1792)              0         
                                                                 
 batch_normalization_10 (Bat  (None, 1792)             7168      
 chNormalization)                                                
                                                                 
 dense_15 (Dense)            (None, 512)               918016    
                                                                 
 dropout_11 (Dropout)        (None, 512)               0         
                                                      

In [ ]:
history1 = model1.fit(ds_train, epochs=10, validation_data=ds_val, callbacks=[model_checkpoint_callback])

Epoch 1/10
27/27 [==============================] - 234s 8s/step - loss: 23.5953 - my_rmse: 23.5764 - val_loss: 20.6890 - val_my_rmse: 20.7217
Epoch 2/10
27/27 [==============================] - 61s 2s/step - loss: 22.4175 - my_rmse: 22.4110 - val_loss: 20.8383 - val_my_rmse: 20.8830
Epoch 3/10
27/27 [==============================] - 61s 2s/step - loss: 22.1723 - my_rmse: 22.1534 - val_loss: 20.8144 - val_my_rmse: 20.8582
Epoch 4/10
27/27 [==============================] - 61s 2s/step - loss: 21.7790 - my_rmse: 21.7743 - val_loss: 20.7170 - val_my_rmse: 20.7386
Epoch 5/10
27/27 [==============================] - 61s 2s/step - loss: 21.8652 - my_rmse: 21.8341 - val_loss: 20.8211 - val_my_rmse: 20.8653
Epoch 6/10
27/27 [==============================] - 61s 2s/step - loss: 21.5731 - my_rmse: 21.5663 - val_loss: 20.7946 - val_my_rmse: 20.8372
Epoch 7/10
27/27 [==============================] - 61s 2s/step - loss: 21.4041 - my_rmse: 21.3748 - val_loss: 21.2453 - val_my_rmse: 21.3032
Epoch

In [ ]:
np.save(f"/content/drive/MyDrive/sjsu/255 project/code/zzj/EfficientNetB4_improved/efficientNetb4_hist.npy", history1.history)

In [ ]:
model1.evaluate(ds_test)

8/8 [==============================] - 15s 2s/step - loss: 21.8268 - my_rmse: 21.8021


[21.826765060424805, 21.80210304260254]

In [ ]:
model2.evaluate(x=new_x_test, y=y_dense_test)

62/62 [==============================] - 0s 2ms/step - loss: 0.6666 - rmse: 0.2116 - my_rmse: 20.9768


[0.6666433811187744, 0.21163536608219147, 20.976795196533203]

In [ ]:
np.save(f"/content/drive/MyDrive/sjsu/255 project/code/zzj/EfficientNetB4_improved/new_mlp_hist.npy", history2.history)